# Exercises XP: W8_D3

# XP — LangChain + LangGraph Agent (with Tavily & LangSmith)

**What you'll learn**
- Set up LangChain and LangSmith for local development and tracing.
- Configure environment variables securely in Python.
- Integrate third-party tools (Tavily) into a LangChain workflow.
- Select and run a Mistral chat model via LangChain.
- Bind tools to a model and inspect tool calls.
- Build a ReAct-style agent using LangGraph prebuilt utilities.
- Stream messages and debug multi-step agent runs.

**Environment**
- Local Jupyter Notebook (Windows) with a dedicated virtual environment and kernel.
- Code cells have English titles and line-by-line comments. Narrative explanations are concise.

---
## Exercises
1) Setup & Environment Configuration  
2) Define Tools (Tavily)  
3) Using Language Models (Mistral)  
4) Tool Binding & Response Inspection  
5) Create the Agent (LangGraph)  
6) Run the Agent  
7) Streaming Messages


## Environment Setup & Dependency Installation

In [18]:
# Description: Install required packages for LangChain, LangGraph, Tavily, and Mistral.

import sys
import subprocess

def pip_install(spec: str):
    """Install a pip package spec quietly and upgrade if needed."""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-qU", *spec.split()])

# Install core LangChain and community tools
pip_install("langchain langchain-community")

# Install Mistral integration for LangChain
pip_install("langchain-mistralai")

# Install Tavily API client
pip_install("tavily-python")

# Install LangGraph (core) and prebuilt agents utilities
pip_install("langgraph langgraph-prebuilt")

# Install LangSmith client for tracing and debugging
pip_install("langsmith")

# Install dotenv helper for loading environment variables from .env files
pip_install("python-dotenv")

print("✅ Dependencies installed.")

✅ Dependencies installed.


## Verify Imports & Print Versions

In [20]:
# Description: Check installed package versions via PyPI metadata and confirm imports work.

import importlib
from importlib import metadata

def package_version(dist_name: str):
    """Return installed PyPI distribution version or 'not installed'."""
    try:
        return metadata.version(dist_name)
    except metadata.PackageNotFoundError:
        return "not installed"

versions = {
    "langchain": package_version("langchain"),
    "langchain-community": package_version("langchain-community"),
    "langchain-mistralai": package_version("langchain-mistralai"),
    "tavily-python": package_version("tavily-python"),
    "langgraph": package_version("langgraph"),
    "langsmith": package_version("langsmith"),
    "python-dotenv": package_version("python-dotenv"),
}

print("📦 Installed package versions:\n", versions)

print("\n--- Import smoke tests ---")
try:
    from langchain_mistralai import ChatMistralAI
    print("✅ Import ok: ChatMistralAI")
except Exception as e:
    print("❌ Import failed: ChatMistralAI ->", e)

try:
    from langchain_community.tools.tavily_search import TavilySearchResults
    print("✅ Import ok: TavilySearchResults")
except Exception as e:
    print("❌ Import failed: TavilySearchResults ->", e)

try:
    import langgraph
    print("✅ Import ok: langgraph")
    import langgraph.prebuilt
    print("✅ Import ok: langgraph.prebuilt")
except Exception as e:
    print("❌ Import failed: langgraph or langgraph.prebuilt ->", e)

try:
    from dotenv import load_dotenv
    print("✅ Import ok: dotenv.load_dotenv")
except Exception as e:
    print("❌ Import failed: dotenv ->", e)

📦 Installed package versions:
 {'langchain': '0.3.27', 'langchain-community': '0.3.27', 'langchain-mistralai': '0.2.11', 'tavily-python': '0.7.10', 'langgraph': '0.6.4', 'langsmith': '0.4.13', 'python-dotenv': '1.1.1'}

--- Import smoke tests ---
✅ Import ok: ChatMistralAI
✅ Import ok: TavilySearchResults
✅ Import ok: langgraph
✅ Import ok: langgraph.prebuilt
✅ Import ok: dotenv.load_dotenv


## Configure LangSmith & Tavily (Secure Prompts)

In [17]:
# Description: Set environment variables securely at runtime using getpass.

import os
import getpass

# Enable LangSmith tracing
# Note: LangSmith will only log traces if LANGSMITH_TRACING is "true".
os.environ["LANGSMITH_TRACING"] = "true"

# Prompt for LangSmith API key only if not already set in the environment
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API Key: ")

# Prompt for Tavily API key only if not already set
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key: ")

# Quick checks
assert os.getenv("LANGSMITH_TRACING") == "true", "LangSmith tracing must be 'true'."
assert os.getenv("LANGSMITH_API_KEY"), "Missing LANGSMITH_API_KEY."
assert os.getenv("TAVILY_API_KEY"), "Missing TAVILY_API_KEY."

print("✅ Environment configured successfully.")

LangSmith API Key:  ········
Tavily API Key:  ········


✅ Environment configured successfully.


## Exercise 2 — Define Tools (Tavily)

Goal:
- Import TavilySearchResults from `langchain_community.tools`.
- Instantiate the search tool.
- Run a test query and assemble the `tools` list.

### Tavily Tool Import & Smoke Test

In [22]:
# Description: Import TavilySearchResults, run a sample query, and assemble a tools list.

# Import the Tavily search tool from LangChain community
from langchain_community.tools.tavily_search import TavilySearchResults

# Instantiate the search tool (reads TAVILY_API_KEY from environment variables)
search = TavilySearchResults()

# Define a small helper to safely truncate long strings for printing
def preview(text: str, n: int = 600) -> str:
    """Return a truncated preview of 'text' with ellipsis if too long."""
    if not isinstance(text, str):
        return str(text)
    return text if len(text) <= n else text[:n] + " ..."

# Run a quick smoke query to ensure the API key is valid and the tool works
query = "latest advancements in AI-driven healthcare triage"
search_results = search.run(query)

# Print a truncated view for readability
print("Query:", query)
print("Sample results:", preview(search_results))

# Assemble the tools list for later binding with the LLM
tools = [search]
print("✅ Tools ready:", [type(t).__name__ for t in tools])

Query: latest advancements in AI-driven healthcare triage
Sample results: [{'title': 'AI Diagnostics: Revolutionizing Medical Diagnosis in 2025', 'url': 'https://www.scispot.com/blog/ai-diagnostics-revolutionizing-medical-diagnosis-in-2025', 'content': 'The implementation of AI-driven predictive analytics significantly improved patient care and healthcare outcomes. By analyzing historical data and identifying patterns, AI algorithms could forecast patient trajectories with high accuracy, enabling healthcare providers to intervene early, prevent complications, and tailor treatments based on individual patient profiles. [...] Looking ahead, advancements in machine learning, integration with wearable devices, personalized medicine, and autonomous diagnostic systems will continue to expand the possibilities of AI in diagnostic medicine. To stay competitive and deliver the best possible care in this rapidly evolving landscape, healthcare organizations must embrace AI diagnostic solutions th

## Exercise 3 — Using Language Models (Mistral)

Goal:
- Ensure your Mistral API key is set.
- Initialize a Mistral chat model via LangChain.
- Send a simple `HumanMessage` and print the response.

Two equivalent paths are shown:
1) `init_chat_model` helper (simple)
2) `ChatMistralAI` class (explicit)

## Ensure Mistral API Key (Skip if already set)

In [23]:
# Description: Prompt for Mistral API key only if it's missing from the environment.

import os, getpass

if not os.environ.get("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Mistral API Key: ")

assert os.environ.get("MISTRAL_API_KEY"), "Missing MISTRAL_API_KEY."
print("✅ Mistral API key is set.")

Mistral API Key:  ········


✅ Mistral API key is set.


In [28]:
# Title: Install Mistral SDK
# Description: Install the official Mistral client to list available models.

import sys
!{sys.executable} -m pip install -qU mistralai
print("✅ mistralai SDK installed.")

✅ mistralai SDK installed.


## Initialize Mistral Chat (Canonical)

In [33]:
# Title: Initialize Mistral Chat (Canonical)
# Description: Use ChatMistralAI directly (avoid 'provider' issues with init_chat_model).

import os
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage

# Pick a valid model for your account. Common choices in 2025:
# - "mistral-small-latest"
# - "mistral-large-latest"
# - "ministral-3b-latest"
MODEL_NAME = os.environ.get("MISTRAL_MODEL", "mistral-small-latest")

chat_model = ChatMistralAI(
    model=MODEL_NAME,
    api_key=os.environ["MISTRAL_API_KEY"],
)

# Quick test
msg = HumanMessage(content="Tell me a fun fact about space exploration.")
resp = chat_model([msg])
print("✅ ChatMistralAI replied:\n", resp.content)

✅ ChatMistralAI replied:
 Here’s a fun fact about space exploration:

**The Voyager 1 spacecraft, launched in 1977, is the farthest human-made object from Earth and is currently in interstellar space—beyond the influence of our Sun’s solar wind. It carries a golden record with sounds and images of Earth, including greetings in 55 languages, music from different cultures, and even the sound of a kiss!**

Bonus fact: Voyager 1’s power will last until at least 2025, but it will continue drifting through the Milky Way for billions of years—potentially one day being discovered by an alien civilization!

Would you like more space trivia? 🚀


## Raw SDK Sanity Check

In [34]:
# Title: (Optional) Raw SDK Sanity Check
# Description: Confirm your key + model works using the official Mistral SDK v1.

from mistralai import Mistral
client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

sdk_resp = client.chat.complete(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "Say 'hello' in one short sentence."}],
)
print("SDK v1 says:", sdk_resp.choices[0].message.content)

SDK v1 says: "Hello there!"


## Exercise 4 — Tool Binding and Response Inspection

Goal:
- Bind the Tavily tool to the chat model using `.bind_tools`.
- Compare a prompt that doesn't need tools vs. one that should trigger a tool call.
- Inspect `.content` and `.tool_calls` on the response.

## Bind Tools to the Model

In [35]:
# Description: Create a tool-aware model by binding the Tavily tool to ChatMistralAI.

# Pre-reqs:
# - `chat_model` created in Exercise 3 (ChatMistralAI instance)
# - `tools` list created in Exercise 2 (e.g., [TavilySearchResults()])

model_with_tools = chat_model.bind_tools(tools)
print("✅ Tools bound to model.")

✅ Tools bound to model.


## Inspect Responses (No Tool vs. Tool Expected)

In [37]:
# Title: Inspect Responses (No Tool vs. Tool Expected)
# Description: Compare a normal prompt vs. a prompt that should require a web search.
# Note: model_with_tools is a RunnableBinding → use .invoke(...), not a direct call.

from langchain_core.messages import HumanMessage

def print_response(label: str, resp):
    """Pretty-print content and tool_calls for a model response."""
    print(f"\n--- {label} ---")
    # resp is typically an AIMessage; show content and any tool_calls
    print("Content:\n", getattr(resp, "content", "(empty)") or "(empty)")
    print("Tool calls:", getattr(resp, "tool_calls", None))

# Case 1 — Should NOT require a tool (expect: content present, tool_calls empty)
resp_no_tool = model_with_tools.invoke([HumanMessage(content="Give me 3 quick stretches after a long flight.")])
print_response("Case 1 (No tool expected)", resp_no_tool)

# Case 2 — SHOULD require a tool (expect: empty content + a tool call to Tavily)
resp_need_tool = model_with_tools.invoke([HumanMessage(content="Search for the latest AI breakthroughs in healthcare triage.")])
print_response("Case 2 (Tool expected)", resp_need_tool)

# Optional: If a tool call exists, show its name and arguments
if getattr(resp_need_tool, "tool_calls", None):
    tc = resp_need_tool.tool_calls[0]
    print("\nFirst tool call → name:", tc.get("name"))
    print("First tool call → args:", tc.get("args"))


--- Case 1 (No tool expected) ---
Content:
 Sure! Here are three quick stretches you can do after a long flight to help relieve tension and improve circulation:

1. **Neck Stretches:**
   - Sit or stand up straight.
   - Gently tilt your head to one side, bringing your ear towards your shoulder.
   - Hold for 15-30 seconds, then switch sides.
   - Repeat 2-3 times on each side.

2. **Shoulder and Arm Stretches:**
   - Extend one arm across your chest.
   - Use your other hand to gently pull it closer to your chest.
   - Hold for 15-30 seconds, then switch arms.
   - Repeat 2-3 times on each side.

3. **Seated Forward Bend:**
   - Sit on the edge of your seat with your legs extended straight in front of you.
   - Gently reach forward, trying to touch your toes or shins.
   - Hold for 15-30 seconds.
   - Repeat 2-3 times.

These stretches can help alleviate stiffness and improve blood flow after a long flight.
Tool calls: []

--- Case 2 (Tool expected) ---
Content:
 (empty)
Tool calls: 

## Tool Registry & Executor

In [38]:
# Description: Map tool call names to concrete tool instances and execute them safely.

from typing import Any, Dict

# Build a registry from tool name → tool instance.
# Your Tavily tool surfaced as "tavily_search_results_json" in tool_calls, so we register that.
TOOL_REGISTRY: Dict[str, Any] = {
    "tavily_search_results_json": search,  # from Exercise 2
    # You can add more tools here later.
}

def execute_tool_call(tool_call: Dict[str, Any]) -> str:
    """
    Execute a single tool call dict of the form:
    {"name": str, "args": dict, "id": str, "type": "tool_call"}
    Returns the stringified result (safe for ToolMessage).
    """
    name = tool_call.get("name")
    args = tool_call.get("args", {})
    tool = TOOL_REGISTRY.get(name)
    if tool is None:
        return f"[Tool error] Unknown tool: {name}"
    try:
        # Most LangChain tools implement .invoke(**kwargs) or .invoke(dict)
        # We'll pass the args dict directly.
        result = tool.invoke(args)
        # Ensure string output for the follow-up message
        return result if isinstance(result, str) else str(result)
    except Exception as e:
        return f"[Tool error] {name} failed: {e}"

## One‑Step ReAct (Manual Round Trip)

In [39]:
# Description: 1) Ask the model_with_tools; 2) Execute any tool calls; 3) Feed tool results back; 4) Get final answer.

from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from typing import List

def react_once(user_prompt: str) -> str:
    """
    Perform a single ReAct-like iteration:
    - Send HumanMessage to tool-aware model (model_with_tools)
    - Execute requested tool(s)
    - Send ToolMessage(s) back to the model
    - Return the final AI answer
    """
    # 1) Initial request: get an AIMessage that may contain tool_calls
    ai_first: AIMessage = model_with_tools.invoke([HumanMessage(content=user_prompt)])

    tool_calls = getattr(ai_first, "tool_calls", []) or []
    if not tool_calls:
        # No tools needed → return the text directly
        return ai_first.content

    # 2) Execute each tool call and collect ToolMessage(s)
    tool_messages: List[ToolMessage] = []
    for tc in tool_calls:
        tool_result = execute_tool_call(tc)
        # ToolMessage requires the tool_call_id to link result to the right call
        tool_messages.append(
            ToolMessage(
                content=tool_result,
                tool_call_id=tc.get("id", "")  # keep the ID from the AIMessage tool call
            )
        )

    # 3) Follow-up turn: provide the tool results and let the model produce the final answer
    final_ai: AIMessage = model_with_tools.invoke(
        [HumanMessage(content=user_prompt), ai_first, *tool_messages]
    )

    # 4) Return final content
    return final_ai.content

# Quick demo
question = "Find 2 recent breakthroughs in AI for emergency medicine and cite sources."
final_answer = react_once(question)
print("USER:", question, "\n")
print("ASSISTANT (final):\n", final_answer)

USER: Find 2 recent breakthroughs in AI for emergency medicine and cite sources. 

ASSISTANT (final):
 Here are two recent breakthroughs in AI for emergency medicine:

1. **AI Model for Converting Hospital Records into Text**:
   - Researchers at UCLA have developed an AI system called the Multimodal Embedding Model for EHR (MEME). This system converts fragmented electronic health records (EHR) into readable narratives, making it easier for AI models to analyze complex patient histories. This advancement allows for more accurate clinical decision support in emergency settings. The system transforms tabular health data into "pseudonotes" that mimic clinical documentation, enabling AI models designed for text to perform better in emergency medicine scenarios. [Read more here](https://www.uclahealth.org/news/release/ai-model-converts-hospital-records-text-better-emergency).

2. **AI for Rapid Diagnostics in Emergency Departments**:
   - AI algorithms are increasingly being used to analyze

## Inspect Intermediate (Optional)

In [40]:
# Description: Show raw tool call + truncated tool output for transparency/debug.

from langchain_core.messages import HumanMessage

q = "Search 1-2 recent AI techniques improving ER triage, with links."
ai_first = model_with_tools.invoke([HumanMessage(content=q)])

print("First AI message (may include tool calls):")
print("- content:", ai_first.content or "(empty)")
print("- tool_calls:", ai_first.tool_calls)

if ai_first.tool_calls:
    tc0 = ai_first.tool_calls[0]
    preview = execute_tool_call(tc0)
    print("\nExecuted first tool call:", tc0.get("name"), "args:", tc0.get("args"))
    print("Tool preview:", (preview[:700] + " ...") if len(preview) > 700 else preview)

First AI message (may include tool calls):
- content: (empty)
- tool_calls: [{'name': 'tavily_search_results_json', 'args': {'query': 'recent AI techniques improving ER triage'}, 'id': '3qLct1LM9', 'type': 'tool_call'}]

Executed first tool call: tavily_search_results_json args: {'query': 'recent AI techniques improving ER triage'}
Tool preview: [{'title': 'AI-driven triage in emergency departments: A review of benefits ...', 'url': 'https://www.sciencedirect.com/science/article/pii/S1386505625000553', 'content': 'Algorithmic models. At the core of AI-driven triage are machine learning (ML) algorithms capable of analyzing and categorizing patient risk levels based on complex data patterns. Various models are used in AI-driven triage systems, including decision trees, neural networks, and regression models .', 'score': 0.98593}, {'title': 'Use of Artificial Intelligence in Triage in Hospital ...', 'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC11158416/', 'content': 'by S Tyler · 2024

## Exercise 5 — Create the Agent (LangGraph)

Goal:
- Use `create_react_agent` from `langgraph.prebuilt`.
- Build an agent executor with `llm=chat_model` and `tools=tools`.
- Verify the executor object is created successfully.

## Build ReAct Agent Executor (LangGraph Prebuilt)

In [43]:
# Description: Use the correct signature: `model=<chat_model>` instead of `llm=`.

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    model=chat_model,   # ✅ correct keyword
    tools=tools
)

print("✅ Agent executor ready:", type(agent_executor).__name__)

✅ Agent executor ready: CompiledStateGraph


## Sanity Check — Invoke Agent Once

In [44]:
# Description: Send a simple prompt and extract the final AI message content.

from langchain_core.messages import HumanMessage, AIMessage

def run_agent_once(prompt: str) -> str:
    """Invoke the agent with a single HumanMessage and return the last AI message content."""
    state = agent_executor.invoke({"messages": [HumanMessage(content=prompt)]})
    msgs = state.get("messages", [])
    last_ai = next((m for m in reversed(msgs) if isinstance(m, AIMessage)), None)
    return last_ai.content if last_ai else "(no AI content found)"

print(run_agent_once("What is the capital of France?"))

The capital of France is Paris.


## Trigger Tool Use via Agent

In [45]:
# Description: Ask something that should require Tavily; the agent will decide to call the tool.

answer = run_agent_once("Find 2 recent breakthroughs in AI for emergency medicine and cite sources.")
print(answer)

[{'type': 'text', 'text': 'Here are two recent breakthroughs in AI for emergency medicine:\n\n1. **AI for Rapid Diagnostics**:\n   - AI algorithms are being used to analyze medical imaging such as chest X-rays, CT scans, and MRIs. This rapid diagnostic capability is particularly useful in emergency departments where quick and accurate diagnosis is crucial for conditions like strokes or aortic dissections. AI can help in quickly identifying critical conditions, leading to faster interventions and potentially saving lives '}, {'type': 'reference', 'reference_ids': ['EMRA']}, {'type': 'text', 'text': '.\n\n2. **AI for Operational Efficiency**:\n   - AI tools are being developed to manage and streamline various operational tasks in emergency departments. These tools can assist with patient triage, clinical presentation diagnosis, mortality prediction, and clinical decision-making. By automating routine tasks such as documentation and data retrieval, AI can help clinicians focus more on dir

## Ask Agent with Inline Markdown Links

In [46]:
# Description: Nudge the agent to return inline markdown links, no separate references.

ask = (
    "Find 2 recent breakthroughs in AI for emergency medicine. "
    "Return a short list of 2 bullet points with one-sentence summaries, "
    "and include the source as an inline markdown link at the end of each bullet. "
    "Do not include separate reference blocks."
)

print(run_agent_once(ask))

- **AI-Powered Rapid Diagnostics**: AI algorithms are now capable of rapidly analyzing medical imaging such as chest X-rays, CT scans, and MRIs, facilitating quicker interventions in critical situations like strokes or aortic dissections. [Source](https://www.emra.org/students/newsletter-articles/ai-in-emergency-medicine)

- **Enhanced Triage Systems**: AI applications have been developed to enhance triage systems in emergency departments, improving the efficiency and accuracy of patient prioritization and care. [Source](https://www.sciencedirect.com/science/article/pii/S2688115225000098)


## Normalize Agent Structured Output to Plain Text

In [47]:
# Description: Flatten list[dict(type=...)] into readable string; keep reference IDs as tags.

from typing import Any, List, Dict

def normalize_agent_content(content: Any) -> str:
    """
    Flatten agent structured content (list of {type:'text'|'reference', ...})
    into a readable string. If reference blocks are present, the IDs are appended
    like [EMRA], [emed]. You can later resolve IDs to URLs if you keep a mapping.
    """
    if isinstance(content, list):
        parts: List[str] = []
        for item in content:
            if isinstance(item, dict) and "type" in item:
                if item["type"] == "text":
                    parts.append(item.get("text", ""))
                elif item["type"] == "reference":
                    ids = item.get("reference_ids", [])
                    if ids:
                        parts.append(" " + " ".join(f"[{rid}]" for rid in ids))
            else:
                parts.append(str(item))
        return "".join(parts).strip()
    return str(content)

# Demo on your last output (replace `your_content` with the AI message content you captured)
your_content = [
 {'type': 'text', 'text': 'Here are two recent breakthroughs in AI for emergency medicine:\n\n1. **AI for Rapid Diagnostics**:\n   - AI algorithms are being used to analyze medical imaging such as chest X-rays, CT scans, and MRIs. This rapid diagnostic capability is particularly useful in emergency departments where quick and accurate diagnosis is crucial for conditions like strokes or aortic dissections. AI can help in quickly identifying critical conditions, leading to faster interventions and potentially saving lives '},
 {'type': 'reference', 'reference_ids': ['EMRA']},
 {'type': 'text', 'text': '.\n\n2. **AI for Operational Efficiency**:\n   - AI tools are being developed to manage and streamline various operational tasks in emergency departments. These tools can assist with patient triage, clinical presentation diagnosis, mortality prediction, and clinical decision-making. By automating routine tasks such as documentation and data retrieval, AI can help clinicians focus more on direct patient care, thereby improving the overall efficiency and quality of care in emergency settings '},
 {'type': 'reference', 'reference_ids': ['emed']},
 {'type': 'text', 'text': '.smhs.gwu.edu.\n\nThese advancements highlight the potential of AI to significantly enhance the efficiency and effectiveness of emergency medical care.'}
]

print(normalize_agent_content(your_content))

Here are two recent breakthroughs in AI for emergency medicine:

1. **AI for Rapid Diagnostics**:
   - AI algorithms are being used to analyze medical imaging such as chest X-rays, CT scans, and MRIs. This rapid diagnostic capability is particularly useful in emergency departments where quick and accurate diagnosis is crucial for conditions like strokes or aortic dissections. AI can help in quickly identifying critical conditions, leading to faster interventions and potentially saving lives  [EMRA].

2. **AI for Operational Efficiency**:
   - AI tools are being developed to manage and streamline various operational tasks in emergency departments. These tools can assist with patient triage, clinical presentation diagnosis, mortality prediction, and clinical decision-making. By automating routine tasks such as documentation and data retrieval, AI can help clinicians focus more on direct patient care, thereby improving the overall efficiency and quality of care in emergency settings  [eme

## Exercise 6: Run the Agent

Goal:
- Run stateless queries through the agent.
- Inspect the returned state (messages timeline).
- Extract the final AI response text.

## Helper — Inspect Agent State

In [48]:
# Description: Pretty-print the agent's messages timeline and extract the final AI message content.

from typing import Optional
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage, AnyMessage

def summarize_state(state: dict) -> str:
    """Return a single string summary of message roles/types in the state."""
    msgs = state.get("messages", [])
    lines = []
    for i, m in enumerate(msgs):
        role = m.type if hasattr(m, "type") else m.__class__.__name__
        # AIMessage may carry tool_calls
        tool_meta = ""
        if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
            tool_meta = f" | tool_calls={len(m.tool_calls)}"
        lines.append(f"{i:02d}. {role}{tool_meta}")
    return "\n".join(lines) if lines else "(no messages)"

def get_final_ai_text(state: dict) -> Optional[str]:
    """Get the last AIMessage content from the state's messages list."""
    msgs = state.get("messages", [])
    for m in reversed(msgs):
        if isinstance(m, AIMessage):
            return m.content
    return None

## Run Two Simple Prompts

In [49]:
# Description: Send two prompts that likely don't need tools and inspect state + final text.

from langchain_core.messages import HumanMessage

def run_and_show(prompt: str):
    print(f"\n=== PROMPT ===\n{prompt}\n")
    state = agent_executor.invoke({"messages": [HumanMessage(content=prompt)]})
    print("=== STATE SUMMARY ===")
    print(summarize_state(state))
    print("\n=== FINAL ANSWER ===")
    print(get_final_ai_text(state) or "(no AI content)")

# Example 1
run_and_show("What is the capital of France?")

# Example 2
run_and_show("Tell me a short joke about robots (one sentence).")


=== PROMPT ===
What is the capital of France?

=== STATE SUMMARY ===
00. human
01. ai

=== FINAL ANSWER ===
The capital of France is Paris.

=== PROMPT ===
Tell me a short joke about robots (one sentence).

=== STATE SUMMARY ===
00. human
01. ai

=== FINAL ANSWER ===
Why did the robot bring a ladder to the bar? Because it heard the drinks were on the house!


## Trigger Tool Use & Inspect Timeline

In [50]:
# Description: Ask something that should require web search; the agent should call Tavily under the hood.

from langchain_core.messages import AIMessage

prompt = "Find 2 recent breakthroughs in AI for emergency medicine and cite sources."
state = agent_executor.invoke({"messages": [HumanMessage(content=prompt)]})

print("=== STATE SUMMARY ===")
print(summarize_state(state))

print("\n=== RAW TIMELINE (truncated) ===")
for i, m in enumerate(state.get("messages", [])):
    label = m.type if hasattr(m, "type") else m.__class__.__name__
    content = getattr(m, "content", "")
    if isinstance(content, list):
        # Some tool/agent messages can be structured lists; show a short preview
        preview = str(content)[:300] + (" ..." if len(str(content)) > 300 else "")
    else:
        preview = (content or "")[:300] + (" ..." if content and len(content) > 300 else "")
    print(f"{i:02d}. {label}: {preview}")

print("\n=== FINAL ANSWER ===")
print(get_final_ai_text(state) or "(no AI content)")

=== STATE SUMMARY ===
00. human
01. ai | tool_calls=1
02. tool
03. ai

=== RAW TIMELINE (truncated) ===
00. human: Find 2 recent breakthroughs in AI for emergency medicine and cite sources.
01. ai: 
02. tool: [{"title": "AI frontiers in emergency care: the next evolution of nursing ...", "url": "https://www.frontiersin.org/journals/public-health/articles/10.3389/fpubh.2024.1439412/full", "content": "5. Han, R, Acosta, JN, Shakeri, Z, Ioannidis, JPA, Topol, EJ, and Rajpurkar, P. Randomised controlled tria ...
03. ai: [{'type': 'text', 'text': 'Recent breakthroughs in AI for emergency medicine include:\n\n1. **AI in Rapid Diagnostics and Imaging Analysis**:\n   AI algorithms have been developed to quickly and accurately analyze medical imaging such as chest X-rays, CT scans, and MRIs. This capability is particula ...

=== FINAL ANSWER ===
[{'type': 'text', 'text': 'Recent breakthroughs in AI for emergency medicine include:\n\n1. **AI in Rapid Diagnostics and Imaging Analysis**:\n   AI al

## Extract Sources Heuristic (Robust)

In [52]:
# Title: Extract Sources Heuristic (Robust)
# Description: Normalize structured AI content to plain text before applying regex for URLs.

import re
from typing import Any, List, Dict

def normalize_agent_content(block: Any) -> str:
    """Flatten structured content (list of {type:'text'|'reference'}) into a readable string."""
    if isinstance(block, list):
        parts: List[str] = []
        for item in block:
            if isinstance(item, dict) and "type" in item:
                if item["type"] == "text":
                    parts.append(item.get("text", ""))
                elif item["type"] == "reference":
                    ids = item.get("reference_ids") or []
                    if ids:
                        parts.append(" " + " ".join(f"[{rid}]" for rid in ids))
            else:
                parts.append(str(item))
        return "".join(parts).strip()
    return str(block) if block is not None else ""

# Get final AI content from previous `state`
raw_final = get_final_ai_text(state)  # may be list/dicts or str
final_text = normalize_agent_content(raw_final)

# Now it's safe to run regex
urls = re.findall(r"https?://\S+", final_text)
if urls:
    print("Detected URLs:")
    for u in urls:
        print("-", u)
else:
    print("No URLs detected in final text.")

No URLs detected in final text.


## Ask Agent with Mandatory Inline Links

In [53]:
# Title: Ask Agent with Mandatory Inline Links
# Description: Force the agent to include clickable markdown links in the final bullets.

ask = (
    "Find 2 recent breakthroughs in AI for emergency medicine. "
    "Return exactly 2 bullet points. For EACH bullet, include ONE inline markdown link "
    "to the primary source (e.g., [Title](https://...)). "
    "Do NOT use separate reference blocks."
)
print(run_agent_once(ask))

- **AI for Predicting Sepsis**: Researchers have developed an AI model that can predict sepsis in patients up to two days before clinical onset, potentially saving countless lives. [Source](https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2783321)

- **AI-Powered Stroke Diagnosis**: A new AI system has been shown to diagnose strokes with high accuracy by analyzing CT scans, enabling faster treatment decisions. [Source](https://www.nature.com/articles/s41746-023-00852-4)


## Extract URLs from Tool Messages

In [57]:
# Description: Parse tool outputs in the agent state and collect any 'url' fields.

import json, ast
from typing import List
from langchain_core.messages import ToolMessage

def extract_urls_from_tools(state) -> List[str]:
    """Walk tool messages in the state and collect URL fields from JSON-ish content."""
    urls = []
    for m in state.get("messages", []):
        if isinstance(m, ToolMessage):
            content = m.content
            data = None
            # Tool content can be a Python list/dict, or a JSON string, or a repr string.
            if isinstance(content, (list, dict)):
                data = content
            elif isinstance(content, str):
                # Try JSON first, then Python literal, else fallback regex
                try:
                    data = json.loads(content)
                except Exception:
                    try:
                        data = ast.literal_eval(content)
                    except Exception:
                        data = None
            # Walk the structure to find 'url' keys
            def walk(x):
                if isinstance(x, dict):
                    if "url" in x and isinstance(x["url"], str):
                        urls.append(x["url"])
                    for v in x.values():
                        walk(v)
                elif isinstance(x, list):
                    for it in x:
                        walk(it)
            if data is not None:
                walk(data)
    # Deduplicate preserving order
    seen, out = set(), []
    for u in urls:
        if u not in seen:
            seen.add(u)
            out.append(u)
    return out

# Example usage on the last state you computed:
tool_urls = extract_urls_from_tools(state)
print("Tool URLs found:")
for u in tool_urls:
    print("-", u if u else "(none)")

Tool URLs found:
- https://www.frontiersin.org/journals/public-health/articles/10.3389/fpubh.2024.1439412/full
- https://www.emra.org/students/newsletter-articles/ai-in-emergency-medicine
- https://www.sciencedirect.com/science/article/pii/S2688115225000098
- https://pmc.ncbi.nlm.nih.gov/articles/PMC11874537/
- https://emed.smhs.gwu.edu/news/ai-may-allow-physicians-regain-their-humanity


## Merge Final Answer with Tool URLs

In [56]:
# Title: Merge Final Answer with Tool URLs
# Description: Print the final answer then append the discovered URLs.

final_text = normalize_agent_content(get_final_ai_text(state))
print(final_text, "\n")
if tool_urls:
    print("Sources:")
    for u in tool_urls[:3]:
        print("-", u)
else:
    print("No tool URLs available.")

Recent breakthroughs in AI for emergency medicine include:

1. **AI in Rapid Diagnostics and Imaging Analysis**:
   AI algorithms have been developed to quickly and accurately analyze medical imaging such as chest X-rays, CT scans, and MRIs. This capability is particularly beneficial in emergency departments where rapid diagnostics can lead to quicker interventions for critical conditions like strokes or aortic dissections. This advancement helps in reducing the time taken to initiate necessary treatments for acute patients  [emra] [emed_gwu].

2. **Enhancing Triage and Predictive Analytics**:
   AI applications are being used to enhance triage systems, predict disease-specific risks, and forecast patient decompensation. These tools can estimate staffing needs and interpret imaging findings, thereby improving the efficiency and accuracy of emergency care. AI-driven triage systems can prioritize patient care based on the severity of their conditions, ensuring that the most critical case

 ## Exercise 7 — Streaming Messages

Goal:
- Stream incremental updates from the agent.
- Display partial messages as they arrive.
- Optionally capture the final AI message for later use.

## Stream Agent Messages (Auto Mode)

In [58]:
# Description: Iterate streaming updates from the agent and pretty-print as they arrive.

from langchain_core.messages import HumanMessage, AIMessage

prompt = "List three recent AI papers on medical imaging with one-sentence summaries."

stream = agent_executor.stream(
    {"messages": [HumanMessage(content=prompt)]},
    stream_mode="auto"  # lets the agent decide how to chunk/flush
)

for step in stream:
    msgs = step.get("messages") or []
    if not msgs:
        continue
    msg = msgs[0]
    # Prefer pretty_print when available
    try:
        msg.pretty_print()
    except Exception:
        content = getattr(msg, "content", "")
        print(content if content else "(no content)")

## Stream Agent Messages (Values Mode) — Always Capture Final

In [61]:
# Description: Use stream_mode="values" to ensure we receive the final state even if the agent ends on a tool call.

from langchain_core.messages import HumanMessage, AIMessage

prompt = "Give 2-3 recent AI methods improving ER triage; include short summaries and inline links."

final_state = None

stream = agent_executor.stream(
    {"messages": [HumanMessage(content=prompt)]},
    stream_mode="values"  # <-- ensures the iterator yields the final state
)

for step in stream:
    # step is a dict of the agent's current "values" (final state at the end)
    # Some intermediate events may not have "messages", so guard access.
    final_state = step  # Keep the last yielded values as the final state
    msgs = step.get("messages") or []
    if not msgs:
        continue
    # Print any incremental message content if present
    m = msgs[-1]  # Usually the most recent message
    content = getattr(m, "content", "")
    try:
        m.pretty_print()
    except Exception:
        print(content if content else "(no content chunk)")

print("\n=== FINAL STATE SUMMARY ===")
def summarize_state(state: dict) -> str:
    from langchain_core.messages import AIMessage
    lines = []
    for i, m in enumerate(state.get("messages", [])):
        role = getattr(m, "type", m.__class__.__name__)
        tool_meta = f" | tool_calls={len(m.tool_calls)}" if isinstance(m, AIMessage) and getattr(m, "tool_calls", None) else ""
        lines.append(f"{i:02d}. {role}{tool_meta}")
    return "\n".join(lines) if lines else "(no messages)"
print(summarize_state(final_state or {}))

# Extract final AI text (may be structured)
from langchain_core.messages import AIMessage
def get_final_ai_text(state: dict):
    msgs = state.get("messages", [])
    for m in reversed(msgs):
        if isinstance(m, AIMessage):
            return m.content
    return None

final_ai_content = get_final_ai_text(final_state or {})  # could be str or structured list

print("\n=== FINAL (raw) ===")
print(final_ai_content if final_ai_content else "(no final AI content)")

================================ Human Message =================================

Give 2-3 recent AI methods improving ER triage; include short summaries and inline links.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (WY6wqQHJf)
 Call ID: WY6wqQHJf
  Args:
    query: recent AI methods improving ER triage
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Improving triage performance in emergency departments ...", "url": "https://pmc.ncbi.nlm.nih.gov/articles/PMC11575054/", "content": "por BM Porto · 2024 · Mencionado por 25 — Studies suggest that Machine Learning (ML) and Natural Language Processing (NLP) could enhance triage accuracy and consistency.", "score": 0.6778372}, {"title": "AI-driven triage in emergency departments: A review of benefits ...", "url": "https://pubmed.ncbi.nlm.nih.gov/39965433/", "content": "Conclusion:\nAI-dri

## Normalize Final Content (If Structured)

In [62]:
# Description: Flatten structured content (list of {type:'text'|'reference'}) into readable text.

from typing import Any, List, Dict

def normalize_agent_content(block: Any) -> str:
    """Flatten structured content into a readable string."""
    if isinstance(block, list):
        parts: List[str] = []
        for item in block:
            if isinstance(item, dict) and "type" in item:
                if item["type"] == "text":
                    parts.append(item.get("text", ""))
                elif item["type"] == "reference":
                    ids = item.get("reference_ids") or []
                    if ids:
                        parts.append(" " + " ".join(f"[{rid}]" for rid in ids))
            else:
                parts.append(str(item))
        return "".join(parts).strip()
    return str(block) if block is not None else ""

print("\n=== FINAL (normalized) ===")
print(normalize_agent_content(final_ai_content))


=== FINAL (normalized) ===
Here are 2-3 recent AI methods improving ER triage, along with short summaries and links for further reading:

1. **Machine Learning (ML) and Natural Language Processing (NLP)**
   - **Summary**: Studies suggest that Machine Learning (ML) and Natural Language Processing (NLP) could enhance triage accuracy and consistency by analyzing vast amounts of patient data quickly and accurately. These technologies help in improving decision-making processes in emergency departments.
   - **Link**: [Improving triage performance in emergency departments](https://pmc.ncbi.nlm.nih.gov/articles/PMC11575054/)

2. **AI-driven Triage Systems**
   - **Summary**: AI-driven triage systems have the potential to transform emergency department (ED) operations by enhancing efficiency, improving patient outcomes, and supporting healthcare professionals in high-pressure environments. These systems optimize resource allocation, reduce wait times, and improve patient prioritization.
   

## Module Deliverables Summary

In [63]:
# Description: Quick check of which exercises have been completed successfully.

from IPython.display import display, HTML

# Liste des exercices avec leur statut
deliverables = [
    {"Exercise": "Exercise 1", "Status": "✅ Done"},
    {"Exercise": "Exercise 2", "Status": "✅ Done"},
    {"Exercise": "Exercise 3", "Status": "✅ Done"},
    {"Exercise": "Exercise 4", "Status": "✅ Done"},
    {"Exercise": "Exercise 5", "Status": "✅ Done"},
    {"Exercise": "Exercise 6 (optional)", "Status": "✅ Done"},
    {"Exercise": "Exercise 7", "Status": "✅ Done"},
]

# Création d'un tableau HTML simple
html_table = "<table style='border-collapse: collapse; width: 50%;'>"
html_table += "<tr><th style='border: 1px solid black; padding: 8px;'>Exercise</th><th style='border: 1px solid black; padding: 8px;'>Status</th></tr>"
for row in deliverables:
    html_table += f"<tr><td style='border: 1px solid black; padding: 8px;'>{row['Exercise']}</td><td style='border: 1px solid black; padding: 8px;'>{row['Status']}</td></tr>"
html_table += "</table>"

display(HTML(html_table))

Exercise,Status
Exercise 1,✅ Done
Exercise 2,✅ Done
Exercise 3,✅ Done
Exercise 4,✅ Done
Exercise 5,✅ Done
Exercise 6 (optional),✅ Done
Exercise 7,✅ Done


## 📌 Module Conclusion

Throughout these exercises, I successfully explored multiple aspects of integrating AI models, tools, and agents using LangChain, LangGraph, and external APIs.  
Here are the main takeaways from each step:

1. **Environment Setup & Package Validation**  
   - Installed and verified required packages (`langchain`, `langgraph`, `tavily-python`, `langchain-mistralai`, `python-dotenv`).
   - Successfully configured API keys for LangSmith, Tavily, and Mistral.

2. **LangChain Basic Chat**  
   - Created a simple chat interface with Mistral AI.
   - Confirmed that the model responds appropriately to general user queries.

3. **Tool Integration**  
   - Linked Tavily search API to the chat model.
   - Verified correct behavior: the model decides when to call a tool vs. when to answer directly.

4. **Mini Pipeline**  
   - Built a minimal workflow where the LLM triggers a web search tool, retrieves information, and formats the response.

5. **Agent Creation with LangGraph**  
   - Created a reactive agent using `create_react_agent`.
   - Tested tool invocation within an agent context for multi-step reasoning.

6. **Optional – Citation & URL Extraction**  
   - Implemented logic to detect URLs in final AI outputs for reference purposes.
   - Confirmed ability to handle both presence and absence of URLs.

7. **State Tracking & Final Output Capture**  
   - Captured AI conversation states, tool calls, and final normalized outputs.
   - Validated complete end-to-end functionality of the pipeline.

---

### 🎯 Overall Outcome
This module provided hands-on experience in:
- Connecting multiple APIs within an AI workflow.
- Building agents capable of reasoning and deciding when to use external tools.
- Managing conversation state and structured outputs.

I am now ready to **adapt this workflow to custom projects** — for example, building a domain-specific chatbot (such as for autism support) that can search, retrieve, and respond with relevant, accurate information.
